In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [2]:
def Cohen_d(group1, group2):

    # Compute Cohen's d.

    # group1: Series or NumPy array
    # group2: Series or NumPy array

    # returns a floating point number 

    diff = np.mean(group1) - np.mean(group2)

    n1, n2 = len(group1), len(group2)
    var1 = np.var(group1)
    var2 = np.var(group2)

    # Calculate the pooled threshold as shown earlier
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    return d

## Import Data

In [3]:
import data_cleaning

In [4]:
clean_data = data_cleaning.full_clean()

In [ ]:
clean_data.head(2)

## Preliminary Analysis 

In [5]:
sig_procs = []
sig_cohens = []

for i in tqdm_notebook(clean_data.drg_definition.unique()):
    metro = clean_data.loc[(clean_data.csi == '1') & (clean_data.drg_definition == i)]['average_covered_charges']
    non_metro = clean_data.loc[(clean_data.csi != '1') & (clean_data.drg_definition == i)]['average_covered_charges']
    if st.ttest_ind(metro, non_metro, equal_var=False)[1] < 0.05:
        sig_procs.append(i)
        sig_cohens.append(Cohen_d(metro, non_metro))
        
relevant_codes = pd.DataFrame([pd.Series(sig_procs, name='sig_procs'), pd.Series(sig_cohens, name='cohens_d')]).T

relevant_codes.sig_procs = relevant_codes.sig_procs.map(lambda x: x[0:3]).astype(int)

drg = pd.read_csv('data/DRG.csv')
drg['group_size'] = drg.Code_End-drg.Code_Start

drg_d = drg.to_dict('records')

code_ref = []
for i in range(0,len(drg_d)):
    for c in relevant_codes.sig_procs:
        if drg_d[i]['Code_Start'] <= c <= drg_d[i]['Code_End']:
            code_ref.append({'sig_procs': c,'Code_Start':drg_d[i]['Code_Start'],'Code_End':
                             drg_d[i]['Code_End'],'Group':drg_d[i]['Description']})
code_map = pd.DataFrame.from_dict(code_ref)
code_map['cohens_d'] = relevant_codes.cohens_d

cum_codes = code_map.groupby('Group').size().reset_index()
cum_cohen = code_map.groupby('Group').cohens_d.sum().reset_index().cohens_d
# cum_codes.columns = ['Description', 'num_sig_procs']

cum_codes['sum_effect'] = cum_cohen
# cum_codes['avg_effect'] = drg.sum_effect/drg.group_size

cum_codes.columns=['Description', 'num_sig_procs', 'sum_effect']

cum_codes = cum_codes.merge(drg, on='Description')
cum_codes['avg_effect'] = cum_codes.sum_effect/cum_codes.group_size
cum_codes['code'] = cum_codes.Description.apply(lambda x: x[0:6])

/home/peter/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/peter/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
plt.figure(figsize=(14,7))
sns.barplot('code', 'avg_effect', data=cum_codes, palette='deep')
plt.xticks(rotation=90)
plt.ylabel('Average Effect')
plt.xlabel('Diagnosis Group')

In [ ]:
proc_group_desc = proc_sum.Description.map(lambda x: x[8:])

In [ ]:
dng_group_code

In [ ]:
dng_info = pd.DataFrame([dng_group_code, proc_group_desc]).T

In [ ]:
dng_info.columns = ['Code', 'Description']

In [ ]:
dng_info.Description = dng_info.Description.map(lambda x: x.title())

In [ ]:
dng_info.to_csv('.code_describe.csv')

In [ ]:
proc_sum['avg_cohen_d'] = proc_sum.sum_cohen_d/proc_sum.group_size

In [ ]:
proc_sum

In [ ]:
type(drg_d[0]['Code_Start'])

In [ ]:
most_sig_proc = 0

for i in sig_procs:
    largest_cohen_d = 0
    metro = clean_data.loc[clean_data.drg_definition == i]['average_covered_charges']
    non_metro = clean_data.loc[clean_data.drg_definition != i]['average_covered_charges']
    if Cohen_d(metro_charges, non_metro_charges) > largest_cohen_d:
         most_sig_proc = i

In [ ]:
most_sig_proc

In [ ]:
metro = clean_data.loc[clean_data.drg_definition == '975 - HIV W MAJOR RELATED CONDITION W CC']['average_covered_charges']
non_metro = clean_data.loc[clean_data.drg_definition != '975 - HIV W MAJOR RELATED CONDITION W CC']['average_covered_charges']

sns.distplot(metro_charges, label='metro')
sns.distplot(non_metro_charges, label='non_metro')
plt.legend()

In [ ]:
metro_means_ = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(metro, size=100))
    metro_means.append(a_mean)

non_metro_means = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(non_metro, size=100))
    non_metro_means.append(a_mean)
    
sns.distplot(metro_means, label='metro')
sns.distplot(non_metro_means, label='non_metro')
plt.legend()
plt.x_label = 'Average Hospital Charges'

### Total Hospital Charges

In [ ]:
metro_charges = clean_data.loc[clean_data['csi'] == '1']['total_discharges']

In [ ]:
non_metro_charges = clean_data.loc[clean_data['csi'] != '1']['total_discharges']

In [ ]:
sns.distplot(metro_charges, label='metro')
sns.distplot(non_metro_charges, label='non_metro')
plt.legend()

In [ ]:
st.ttest_ind(metro_charges, non_metro_charges, equal_var=False)

In [ ]:
Cohen_d(metro_charges, non_metro_charges)

In [ ]:
metro_means = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(metro_charges, size=100))
    metro_means.append(a_mean)

non_metro_means = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(non_metro_charges, size=100))
    non_metro_means.append(a_mean)

sns.distplot(metro_means, label='metro')
sns.distplot(non_metro_means, label='non_metro')
plt.legend()
plt.x_label = 'Average Hospital Charges'

### Patient Deductible/Charges

In [ ]:
metro_deductible = clean_data.loc[clean_data['CSI'] == '1']['deductible']

In [ ]:
non_metro_deductible = clean_data.loc[clean_data['CSI'] != '1']['deductible']

In [ ]:
sns.distplot(metro_deductible, label='metro')
sns.distplot(non_metro_deductible, label='non_metro')
plt.legend()

In [ ]:
metro_deductible_means = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(metro_deductible, size=100))
    metro_deductible_means.append(a_mean)

In [ ]:
non_metro_deductible_means = []

for i in range(0,100000):
    a_mean = np.mean(np.random.choice(non_metro_deductible, size=100))
    non_metro_deductible_means.append(a_mean)

In [ ]:
st.ttest_ind(metro_deductible_means, non_metro_deductible_means, equal_var=False)

In [ ]:
sns.distplot(metro_deductible_means, label='metro')
sns.distplot(non_metro_deductible_means, label='non_metro')
plt.legend()
plt.x_label = 'Deductible'

In [ ]:
Cohen_d(metro_deductible_means, non_metro_deductible_means)